In [ ]:
!pip install Flask

In [ ]:
import openai

# Set your OpenAI API key
openai.api_key = input("Please enter your OpenAI API key: ")

# Function to generate recommendations based on user input using AI
def generate_fruit_recommendations(go_out_to_party, flavor_preference, texture_dislike, price_range):
    # Define the prompt based on user responses
    prompt = f"User preferences:\n\n- Go out to party on weekends: {go_out_to_party}\n- Flavor preference: {flavor_preference}\n- Texture dislike: {texture_dislike}\n- Price range: ${price_range}\n\nBased on these preferences, recommend fruits:"

    # AI completion using OpenAI's GPT model
    completion = openai.Completion.create(
        engine="text-babbage-002",
        prompt=prompt,
        max_tokens=100,
        api_key=openai.api_key
    )

    # Extract recommended fruits from AI's completion
    recommended_fruits = completion.choices[0].text.strip()

    return recommended_fruits

# Gather user input
go_out_to_party = input("Do you go out to party on weekends? (yes or no): ")
flavor_preference = input("What flavors do you like? (cider, sweet, waterlike): ")
texture_dislike = input("What texture do you dislike? (smooth, slimy, rough): ")
price_range_input = input("What price range will you buy drinks for? ($1, $2, $3, $4, $5, $6, $7, $8, $9, $10): ")
while True:
    try:
        price_range = int(price_range_input)
        if price_range < 1 or price_range > 10:
            raise ValueError("Price range must be between 1 and 10")
        break
    except ValueError:
        price_range_input = input("Please enter a valid price range between $1 and $10: ")

# Generate recommendations based on user input using AI
recommended_fruits = generate_fruit_recommendations(go_out_to_party, flavor_preference, texture_dislike, price_range)

# Print the recommended fruits
print("Recommended fruits based on your preferences:", recommended_fruits)


In [ ]:
# Flask API Setup
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/recommend_fruits', methods=['POST'])
def recommend_fruits():
    data = request.json
    go_out_to_party = data.get("go_out_to_party")
    flavor_preference = data.get("flavor_preference")
    texture_dislike = data.get("texture_dislike")
    price_range = data.get("price_range")

    if None in [go_out_to_party, flavor_preference, texture_dislike, price_range]:
        return jsonify({"error": "Missing required fields"}), 400

    recommendations_prompt = generate_fruit_recommendations(go_out_to_party, flavor_preference, texture_dislike, price_range)
    return jsonify({"recommendations": recommendations_prompt})

if __name__ == '__main__':
    app.run(debug=True)